In [1]:
import radarsimpy
print('`RadarSimPy` used in this example is version: ' +
      str(radarsimpy.__version__))

`RadarSimPy` used in this example is version: 11.2.0


# Interferometric Radar

[![Documentations](https://img.shields.io/github/v/tag/radarsimx/radarsimpy?label=Documentation&logo=read-the-docs)](https://radarsimx.github.io/radarsimpy/)
[![Download](https://img.shields.io/github/v/tag/radarsimx/radarsimpy?label=Download&logo=python)](https://radarsimx.com/product/radarsimpy/)

## Introduction

Within radar technology, interferometry encompasses a range of measurement methods wherein the assessment hinges upon the phase shift within the received signal. Particularly, interferometric radar has demonstrated its utility in capturing minuscule vital signs. An interferometric radar designed for monitoring slight vital signs stands as an exemplar of cutting-edge technology. This innovative approach enables the non-invasive and meticulous tracking of essential metrics like heart rate and respiration, all achieved through a contactless methodology. By synergizing radar's sensitivity to even the faintest motions with interferometry's precision in gauging phase discrepancies, this technology showcases a remarkable advancement.

Interferometric radar operates by emitting electromagnetic waves, usually in the microwave frequency range, towards a target area where a person is located. These waves then reflect off the person's body and return to the radar sensor. As the person's body moves due to vital functions like breathing and heartbeat, the reflected waves undergo frequency and phase shifts, which are proportional to these movements.

By carefully analyzing the phase differences between the emitted and received signals, the radar system can extract information about the person's vital signs. This technology is incredibly sensitive, allowing it to detect even the slightest movements associated with breathing and heartbeat. Since it operates from a distance, it offers a non-invasive and unobtrusive method of monitoring vital signs, making it suitable for various applications, including healthcare, wellness monitoring, and security.

In healthcare, interferometric radar for vital sign measurement can be used in hospital settings to continuously monitor patients' conditions without requiring physical contact. It is especially useful for patients who are in critical condition or require constant monitoring, as well as for situations where traditional methods might be uncomfortable or impractical. Moreover, this technology has potential applications in home healthcare and elderly care, enabling remote monitoring and timely intervention when needed.




This is a Doppler (interferometry) radar simulation example based on [`RadarSimPy`](https://radarsimx.com/radarsimx/radarsimpy/).

---


## Radar Model
### Transmitter

Setup the basic transmitter parameters through `Transmitter` module.


### Receiver

Setup the receiver parameters through `Receiver` module.

A single receiver channel is used in this example. The receiver channel is located at [0, 0, 0].

In [2]:
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver

tx = Transmitter(f=24.125e9,
                 t=20,
                 tx_power=10,
                 pulses=1,
                 channels=[dict(location=(0, 0, 0))])

rx = Receiver(fs=20,
              noise_figure=12,
              rf_gain=20,
              baseband_gain=50,
              load_resistor=1000,
              channels=[dict(location=(0, 0, 0))])

radar = Radar(transmitter=tx, receiver=rx)


### Targets

Define a list of targets. There is only one target in this simulaiton. The motion of the target is defined by:

`1.4+1e-3*np.sin(2*np.pi*1*radar.timestamp)`

`radar.timestamp` is the time matrix of the baseband samples.

In [3]:
target = dict(location=(1.4+1e-3*np.sin(2*np.pi*1*radar.timestamp), 0, 0),
              rcs=-10,
              phase=0)

targets = [target]


## Simulate Baseband Signals

Use the `simulator` module to simulate the baseband samples. The user can choose between Python engine `simpy` or C++ engine `simc`.

The output baseband data is a 3-D matrix:

$[channels, pulses, ADC~samples]$

In [4]:
from radarsimpy.simulator import simc

bb_data = simc(radar, targets)
timestamp = bb_data['timestamp']
baseband = bb_data['baseband']


Plot the baseband

In [5]:
import plotly.graph_objs as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=timestamp[0, 0, :],
    y=np.real(baseband[0, 0, :]),
    name='I',
))
fig.add_trace(go.Scatter(
    x=timestamp[0, 0, :],
    y=np.imag(baseband[0, 0, :]),
    name='Q',
))

fig.update_layout(
    title='I/Q Baseband Signals',
    yaxis=dict(title='Amplitude (V)'),
    xaxis=dict(title='Time (s)'),
)

fig.show()


In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.real(baseband[0, 0, :]),
    y=np.imag(baseband[0, 0, :]),
))

fig.update_layout(
    title='I/Q Constellation Plot',
    xaxis=dict(range=[-1, 1], tickformat='.2f', title='In-phase (V)'),
    yaxis=dict(range=[-1, 1],
               tickformat='.2f',
               title='Quadrature (V)',
               scaleanchor="x",
               scaleratio=1),
    height=600,
    width=600,
)

fig.show()


## Phase demodulation

Demodulate the phase of the baseband signal

In [7]:
demod = np.angle(baseband[0, 0, :])


Plot

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=timestamp[0, 0, :],
    y=demod,
))

fig.update_layout(
    title='Phase demodulated',
    xaxis=dict(tickformat='.2f', title='Time (s)'),
    yaxis=dict(tickformat='.2f', title='Phase (rad)'),
)

fig.show()
